In [ ]:
import torchvision.models as models
import torch.nn as nn
from torchinfo import summary
import torch.optim
from tqdm import tqdm
import os
from torch.utils.data import Dataset, DataLoader
import json
from glob import glob
from PIL import Image
import torchvision.transforms.functional as F
from torchvision import transforms
import numpy as np
import numbers
import time
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import csv
import seaborn as sn
import yaml
import pandas as pd
import datetime
from colorama import Fore, Back, Style
from borb.pdf import Document, X11Color, Page, Alignment, SingleColumnLayout, Paragraph, PDF, TableCell, FlexibleColumnWidthTable
from decimal import Decimal
from IPython.display import clear_output
from scipy.interpolate import splrep, BSpline

Il est très difficile (voir impossible) de faire un système déterministe (donc reproduisible) avec pytorch, car il y differentes sources d'aléatoire:
- RNG de python (random)
- RNG de pytorch (pour initialiser les poids des nouvelles couches des modèles et pour le shuffle)
- Le GPU (le GPU optimise sont algorithme à chaque fois ce qui change les résultats)
- RNG numpy
- Algorithmes de pytorch non déterministes
- ...

In [ ]:
data_dir = 'dataset' # name of dataset (with subfolder 'train', 'val', 'test')
seed_results_dir = 'Seed_Results_mobile_base_notime' # name of main directory with results
bias_init = True
bias_init_value = 7.33 # Taille moyenne des rangées

base_model = 'mobile' # in 'res', 'efficient', 'mobile'

# Graines, si False, les RNG du modèle et du shuffle prendrons les valeurs de seeds
Same_model = True
Model_gen_seed = 0 # graine de génération pour générer les modèles (pour avoir les mêmes poids initiaux)

Same_shuffle = False
Shuffle_seed = 0 # graine pour avoir le même shuffle (ordre des images pour l'entrainement)

Time_limit = None # None ou int/float, en heures

In [ ]:
results_pdf = 'results_RMSprop.pdf'
# Seed
seeds = [1] # int: seeds a tester
#for i in range(11):
#    seeds+=[i]

params = {}
# params optimizer
params['optimizer'] = 'RMSprop'                 # str: 'SGD', 'Adadelta', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSprop', ('LBFGS', 'Adagrad' error)
params['lr'] = 0.001                            # float: All
params['weight_decay'] = 0                      # float: All other than LBFGS
params['epsilon'] = 1.0e-06                     # float: Adadelta, Adagrad, Adam, AdamW, NAdam, RAdam, RMSprop
params['rho'] = None                            # float: Adadelta, Adagrad
params['initial_accumulator_value'] = None      # float: Adagrad
params['betas'] = None                          # [float, float]: Adam, AdamW, NAdam, RAdam
params['max_iter'] = None                       # int: LBFGS
params['momentum_decay'] = None                 # float: NAdam
params['alpha'] = 0                             # float: RMSprop
params['momentum'] = 0                          # float: SGD, RMSprop

# params criterion
params['loss_function'] = 'Huber'               # str: 'MAE', 'MSE', 'Huber'
params['delta'] = 3                             # float: Huber

# autres hyper paramètres
params['imgsz'] = 224                           # int or [int, int]: (w,h), minimum 224 (pour resnet)
params['batch_size'] = 64                       # int: maximum 256 avec resnet18 modifier avec 3 couches FC (limite VRAM GPU)
params['epochs'] = 3                            # int
params['freeze'] = False                        # bool
params['earlystop'] = 200                       # None ou int

params['dataset'] = data_dir
if isinstance(params['imgsz'], int):
    params['imgsz']=[params['imgsz'],params['imgsz']]

In [ ]:
def gen_new_model_resnet(freeze, bias_init_value = 0):
    """Génère un model basé sur ResNet18 avec des poids pré-entrainés

    Args:
        freeze (bool): Booléen dictant le gel ou non des couches de convolution du modèle (gel = pas de modifications des poids pendant l'entrainement)
        bias_init_value (float, optional): valeur de bias initial pour la dernière couche. Defaults to 0.

    Returns:
        nn.Module: Le modèle
    """
    model = models.resnet18(weights='IMAGENET1K_V1')

    #modification de la dernière couche
    model.fc = nn.Sequential( 
        nn.Linear(512, 1)
    )

    if bias_init:
        with torch.no_grad():
            model.fc[-1].bias = nn.Parameter(torch.full(model.fc[-1].bias.shape, bias_init_value))

    # gel ou non du reste
    if freeze:
        for param in model.parameters():
            param.requires_grad = False

        for param in model.fc.parameters():
            param.requires_grad = True
    return model


In [ ]:
def gen_new_model_efficientnet(freeze, bias_init_value = 0):
    """Génère un model basé sur EfficientNet_b0 avec des poids pré-entrainés

    Args:
        freeze (bool): Booléen dictant le gel ou non des couches de convolution du modèle (gel = pas de modifications des poids pendant l'entrainement)
        bias_init_value (float, optional): valeur de bias initial pour la dernière couche. Defaults to 0.

    Returns:
        nn.Module: Le modèle
    """
    model = models.efficientnet_b0(weights='IMAGENET1K_V1')

    #modification de la dernière couche
    model.classifier[-1] = nn.Linear(1280,1)

    if bias_init:
        with torch.no_grad():
            model.classifier[-1].bias = nn.Parameter(torch.full(model.fc[-1].bias.shape, bias_init_value))

    # gel ou non du reste
    if freeze:
        for param in model.parameters():
            param.requires_grad = False

        for param in model.classifier.parameters():
            param.requires_grad = True
    return model

In [ ]:
def gen_new_model_mobilenet(freeze, bias_init_value = 0):
    """Génère un model basé sur MobileNet_v3_small avec des poids pré-entrainés

    Args:
        freeze (bool): Booléen dictant le gel ou non des couches de convolution du modèle (gel = pas de modifications des poids pendant l'entrainement)
        bias_init_value (float, optional): valeur de bias initial pour la dernière couche. Defaults to 0.

    Returns:
        nn.Module: Le modèle
    """
    model = models.mobilenet_v3_small(weights='IMAGENET1K_V1')

    #modification de la dernière couche
    model.classifier[-1] = nn.Linear(1024,1)

    if bias_init:
        with torch.no_grad():
            model.classifier[-1].bias = nn.Parameter(torch.full(model.fc[-1].bias.shape, bias_init_value))

    # gel ou non du reste
    if freeze:
        for param in model.parameters():
            param.requires_grad = False

        for param in model.classifier.parameters():
            param.requires_grad = True
    return model

In [ ]:
def gen_new_model_function(base_model):
    """Retourne la fonction qui permer de générer un modèle du type choisi

    Args:
        base_model (str): Nom de l'implémentation du modèle de base pré-entrainé

    Returns:
        function: Fonction qui sert à générer le modèle
    """
    match base_model:
        case 'res':
            return gen_new_model_resnet
        case 'efficient':
            return gen_new_model_efficientnet
        case 'mobile':
            return gen_new_model_mobilenet

In [ ]:
gen_new_model = gen_new_model_function(base_model)

In [ ]:
# algorithmes GPU deterministes
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
class TablePrinter(object):
    "Print a list of dicts as a table"
    def __init__(self, fmt, sep=' ', ul=None):
        """        
        @param fmt: list of tuple(heading, key, width)
                        heading: str, column label
                        key: dictionary key to value to print
                        width: int, column width in chars
        @param sep: string, separation between columns
        @param ul: string, character to underline column label, or None for no underlining
        """
        super(TablePrinter,self).__init__()
        self.fmt   = str(sep).join('{lb}{0}:{1}{rb}'.format(key, width, lb='{', rb='}') for _,key,width in fmt)
        self.head  = {key:heading for heading,key,_ in fmt}
        self.ul    = {key:str(ul)*width for _,key,width in fmt} if ul else None
        self.width = {key:width for _,key,width in fmt}

    def row(self, data):
        return self.fmt.format(**{ k:str(data.get(k,''))[:w] for k,w in self.width.items()})

    def __call__(self, dataList):
        _r = self.row
        res = [_r(data) for data in dataList]
        res.insert(0, _r(self.head))
        if self.ul:
            res.insert(1, _r(self.ul))
        return '\n'.join(res)

In [ ]:
def get_path(params):
    """Donne le nom du chemin à partir des valeurs des paramètres

    Args:
        params (Dict): Dictionnaire des paramètres
    """
    hyp_dir = f'hyp_{params['delta']}\
_{params['lr']}\
_{params['weight_decay']}\
_{params['epsilon']}\
_{params['rho']}\
_{params['initial_accumulator_value']}\
_{params['betas']}\
_{params['max_iter']}\
_{params['momentum_decay']}\
_{params['alpha']}\
_{params['momentum']}'

    result_dir="-".join([f'earlystop_{params['earlystop']}'
                            , f'freeze_{params['freeze']}'
                            , f'imgsz_{params['imgsz']}'
                            , f'batch_{params['batch_size']}'
                            , f'epochs_{params['epochs']}'
                            , params['loss_function']
                            , params['optimizer']
                            , hyp_dir])
    return result_dir

In [ ]:
class MyDataset(Dataset):
    """Classe de dataset custom

    Attributs:
        data_dir (str): Chemin vers les données
        transform (torch.Transform): Transformations appliquées aux données
        image_files (List[nom_image1, ...]): Liste des noms des fichiers des images
        labels (Dict[nom_image1: label, ...]): Dictionnaire de correspondance nom_image-label
    """
    def __init__(self, data_dir, split='train', transform=None):
        """Init

        Args:
            data_dir (str): Chemin vers les données
            split (str, optional): Split du dataset. Defaults to 'train'.
            transform (torch.Transform): Transformations appliquées aux données
        """
        self.data_dir = os.path.join(data_dir, split)
        self.transform = transform
        self.image_files = [file for file in os.listdir(self.data_dir) if file.endswith('.jpg') or file.endswith('.jpeg')]
        self.labels = self.parse_json()

    def __len__(self):
        """Donne la taille du Dataset

        Returns:
            int: taille du dataset
        """
        return len(self.image_files)
    
    def __getitem__(self, idx):
        """Récupère une donnée et son label associé

        Args:
            idx (int): index de la donnée

        Returns:
            Tuple[donnée, label]: Donnée et label
        """
        image_path = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")
        label = self.labels[self.image_files[idx]]
        label = torch.tensor(label, dtype=torch.float32)
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def collate_fn(self, batch: List):
        """Fonction de stacking pour la création d'un batch

        Args:
            batch (List[Tuple]): Liste des Tuples[Tensor[3,imgsz, imgsz], Tensor[1]] avec les tenseurs de l'image et du label

        Returns:
            Tuple[Tensor[Tensor[3,imgsz, imgsz]], List[Tensor[1]]]: Batch
        """
        images_zip, labels_zip = zip(*batch)
        images_batch = torch.stack(images_zip, dim = 0)
        labels_batch = torch.stack(labels_zip, dim = 0)
        return (images_batch, labels_batch)
    
    def parse_json(self):
        """Analyse le json de correspondance nom_image-label

        Returns:
            Dict[nom_image1: label, ...]: Dictionnaire des correspondances
        """
        fname = glob('*.json', dir_fd=self.data_dir)[0]
        with open(os.path.join(self.data_dir, fname)) as json_f:
            labels = json.load(json_f)
        return labels


In [ ]:
def get_criterion(params):
    """Génère la fonction de perte

    Args:
        params (Dict): Dictionnaire des paramètres

    Returns:
        nn.Loss: Fonction de perte
    """
    match params['loss_function']:
        case 'MSE':
            criterion = nn.MSELoss()
        case 'MAE':
            criterion = nn.L1Loss()
        case 'Huber':
            criterion = nn.HuberLoss(delta = params['delta'])
    return criterion

def get_optimizer(model, params):
    """Génère l'optimiseur

    Args:
        model (nn.Module): Modèle
        params (Dict): Dictionnaire des paramètres

    Returns:
        nn.Optim: Optimiseur
    """
    lr = params['lr']
    wd = params['weight_decay']
    eps = params['epsilon']
    rho = params['rho']
    init = params['initial_accumulator_value']
    betas = params['betas']
    max_iter = params['max_iter']
    md = params['momentum_decay']
    alpha = params['alpha']
    momentum = params['momentum']
    
    match params['optimizer']:
        case 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        case 'Adadelta':
            optimizer = torch.optim.Adadelta(model.parameters(), lr = lr, rho = rho, eps=eps, weight_decay=wd)
        case 'Adagrad':
            optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=wd, eps=eps, initial_accumulator_value=init)
        case 'RMSprop':
            optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=wd, eps=eps, alpha=alpha, momentum=momentum)
        case 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
        case 'AdamW':
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
        case 'LBFGS':
            optimizer = torch.optim.LBFGS(model.parameters(), lr=lr, max_iter=max_iter)
        case 'NAdam':
            optimizer = torch.optim.NAdam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas, momentum_decay=md)
        case 'RAdam':
            optimizer = torch.optim.RAdam(model.parameters(), lr=lr, weight_decay=wd, eps=eps, betas=betas)
    return optimizer

def get_dataloaders(params:dict):
    """Génère le dataloader

    Args:
        params (dict): Dictionnaire des paramètres

    Returns:
        torch.Dataloader: Dataloader
    """
    data_dir = params['dataset']
    batch_size = params['batch_size']
    imgsz  = params['imgsz']
    dataloaders = {}
    splits = []
    for split in ['test', 'val', 'train']:
        if split in os.listdir(data_dir):
            splits .append(split)

    data_transforms = transforms.Compose([
            transforms.Resize((imgsz[1], imgsz[0])), # h,w
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalisation demandée par la documentation resnet
        ])           
     
    for split in splits:
        dataset = MyDataset(data_dir = data_dir, split = split, transform = data_transforms)
        # shuffle change l'ordre de passage des données
        dataloaders[split] = DataLoader(dataset=dataset, shuffle=True, batch_size = batch_size, collate_fn = dataset.collate_fn)
    return dataloaders, data_transforms

In [ ]:
def gen_pdf(results, path):
    """Génère un PDF avec un tableau récapitulatif

    Args:
        results (List): Liste des résultats
        path (str): Nom du fichier
    """
    # create document
    pdf = Document()
    
    # add page
    page = Page(Decimal(1500), Decimal(1684))
    m = Decimal(10)
    pdf.add_page(page)
    layout = SingleColumnLayout(page)
    for i in range(0, len(results), 38):
        res = results[i:min(i+38, len(results))]
        table = FlexibleColumnWidthTable(number_of_rows=len(res), number_of_columns=2)
        for name, v in [(k,v) for k,v in res]:
            if isinstance(v, str):
                table.add(TableCell(Paragraph(text=name, font="Helvetica-Bold", font_size=Decimal(12))))
                table.add(TableCell(Paragraph(text=v, font="Helvetica-Bold", font_size=Decimal(12))))
                continue
            table.add(TableCell(Paragraph(name)))
            table.add(            TableCell(
                    Paragraph(str(v), horizontal_alignment=Alignment.CENTERED)
                ))
        table.set_padding_on_all_cells(Decimal(m), Decimal(m), Decimal(m), Decimal(m))
        # set border
        table.set_border_width_on_all_cells(Decimal(0.2))
        layout.add(table)
    with open(path, "wb") as in_file_handle:
        PDF.dumps(in_file_handle, pdf)

In [ ]:
def parse_json_list(path: str):
    """Analyse le json et donne une liste des labels

    Args:
        path (str): Chemin vers le json

    Returns:
        List: Liste des labels
    """
    fname = glob('*.json', dir_fd=path)[0]
    with open(os.path.join(path, fname)) as json_f:
        json_data = json.load(json_f)
        labels = list(json_data.values())
    return labels

def parse_json_dict(path):
    """Analyse le json et donne un dictionnaire

    Args:
        path (str): Chemin vers le json

    Returns:
        Dict: Dictionnaire des correspondances nom_image_label
    """
    fname = glob('*.json', dir_fd=path)[0]
    with open(os.path.join(path, fname)) as json_f:
        labels = json.load(json_f)
    return labels

In [ ]:
def train_step(inputs, targets, model, optimizer, criterion):
    """Une étape d'entrainement

    Args:
        inputs (Tensor[Tensor[3,imgsz, imgsz]]): Images en entrée
        targets (Tensor[Tensor[1]]): Labels cibles
        model (nn.Module): Modèle
        optimizer (Torch.Optim): Optimiseur
        criterion (nn.Loss): Fonction de perte

    Raises:
        ValueError: Explosion de gradient

    Returns:
        float, float, float, int: Métriques
    """
    device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mae_loss = nn.L1Loss()
    model.train()
    for param in model.parameters():
        param.grad = None
    inputs = inputs.to(device)
    targets = targets.to(device)
    try:
        predictions = model(inputs)
        loss = criterion(predictions.squeeze(), targets)
        mae = mae_loss(predictions.squeeze(), targets)
        loss.backward()
        optimizer.step()
        acc = 0
        round_mae = 0
        for i in range(len(predictions)):
            pred = round(predictions[i].item())
            target = int(targets[i].item())
            round_mae += abs(target-pred)
            if pred == target:
                acc +=1
        acc /= len(predictions)
        round_mae /= len(predictions)
    except ValueError as ve:
        raise ValueError(ve)
    del inputs, targets
    torch.cuda.empty_cache()
    return acc, loss.item(), mae.item(), round_mae

@torch.no_grad
def val_step(inputs, targets, model, criterion):
    """Une étape d'évaluation

    Args:
        inputs (Tensor[Tensor[3,imgsz, imgsz]]): Images en entrée
        targets (Tensor[Tensor[1]]): Labels cibles
        model (nn.Module): Modèle
        criterion (nn.Loss): Fonction de perte

    Raises:
        ValueError: Explosion de gradient

    Returns:
        float, float, float, int: Métriques
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mae_loss = nn.L1Loss()
    model.eval()
    inputs = inputs.to(device)
    targets = targets.to(device)
    acc = 0
    round_mae = 0
    try:
        predictions = model(inputs)
        loss = criterion(predictions.squeeze(), targets)
        mae = mae_loss(predictions.squeeze(), targets)
        for i in range(len(predictions)):
            pred = round(predictions[i].item())
            target = int(targets[i].item())
            round_mae += abs(target-pred)
            if pred == target:
                acc +=1
        acc /= len(predictions)
        round_mae /= len(predictions)
    except ValueError as ve:
        raise ValueError(ve)
    
    del inputs, targets
    torch.cuda.empty_cache()
    return acc, loss.item(), mae.item(), round_mae

@torch.no_grad
def get_confusion_matrix(model, path, transform, margin_view):
    """Génère une matrice de confusion

    Args:
        model (nn.Module): Modèle
        path (str): Chemin vers les données
        transform (torch.Optim): Tra,nsformations
        margin_view (int): Marge de valeurs visibles

    Returns:
        np.Array, np.Array, np.Array: Matrice de confusion, indices prédiction et vérité (label)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fnames= glob('*.jpg', dir_fd = path)
    labels = parse_json_dict(path)
    # init matrix
    indices_t = np.linspace(1, max(labels.values()), max(labels.values()))
    indices_p = np.linspace(1, max(labels.values())+margin_view, max(labels.values())+margin_view)
    confusion_matrix = np.zeros((indices_p.size+1, indices_t.size), dtype = np.int16)
    # predictions
    model.eval()
    for fname in fnames:
        image = Image.open(os.path.join(path, fname)).convert("RGB")
        image = transform(image)
        image = torch.unsqueeze(image, dim=0)
        image = image.to(device)
        pred = model(image)
        pred = round(pred[0][0].item())
        if pred in indices_p:
            confusion_matrix[pred-1][labels[fname]-1]+=1
        else:
            confusion_matrix[-1][labels[fname]-1]+=1
    return confusion_matrix, indices_p, indices_t

In [ ]:
# frormat des prints à chaque epoch
fmt = [
    ('Epoch', 'epoch', 8),
    ('Train Loss', 'train loss', 20),
    ('Train Accuracy', 'train acc', 20),
    ('Train MAE', 'train mae', 20),
    ('Train Round MAE', 'train round mae', 20),
    ('Val Loss', 'val loss', 20),
    ('Val Accuracy', 'val acc', 20),
    ('Val MAE', 'val mae', 20),
    ('Val Round MAE', 'val round mae', 20),
    ]

In [ ]:
def training_loop(train_params):
    """Boucle d'entrainement

    Args:
        train_params (Dict): Dictionnaire de paramètres

    Raises:
        ValueError: Explosion de gradient

    Returns:
        float: Meilleure précision
    """
    model = train_params['model']
    num_epochs = train_params['epochs']
    earlystop = train_params['earlystop']
    results_dir = train_params['results_dir']
    dataloaders = train_params['dataloarders']
    optimizer = train_params['optimizer']
    criterion = train_params['criterion']
    data_transforms = train_params['data_transforms']
    if os.path.exists(os.path.join(results_dir, 'results.yaml')):
        print('Already trained')
        with open(os.path.join(results_dir, 'results.yaml'), 'r') as res:
            res_data = yaml.safe_load(res)
            try:
                best_val_accs = res_data['best val acc']
            except KeyError:
                best_val_accs = 0
            training = False
    else:
        training = True

    if training:
        print(Fore.CYAN + results_dir)
        print(os.path.join(os.getcwd(), results_dir) + Fore.RESET)
        # initialisations
        average_training_losses = []
        average_training_accs = []
        average_val_accs = []
        average_val_losses = []
        average_training_maes = []
        average_val_maes = []
        average_training_round_maes = []
        average_val_round_maes = []
        best_val_accs = 0
        best_mae_of_best_acc = float('inf')
        best_round_mae_of_best_acc = float('inf')
        best_epoch = 0
        best_mae_of_best_acc_epoch = 0
        best_round_mae_of_best_acc_epoch = 0
        best_epochs_steps = []
        best_mae_of_best_acc_epochs_steps = []
        best_round_mae_of_best_acc_epochs_steps = []
        best_round_mae_epochs_steps = []
        best_round_mae = float('inf')
        best_acc_of_round_mae = 0
        best_round_mae_epoch = 0
        early_stopping = None

        start_time = time.time()
        with open(os.path.join(results_dir, 'record.tsv'), 'w') as tsvfile:
            writer = csv.writer(tsvfile, delimiter = '\t', lineterminator = '\n')
            writer.writerow(['epoch', 'train_loss', 'train_acc', 'train_mae_loss', 'train_round_mae', 'val_loss', 'val_acc', 'val_mae_loss', 'val_round_mae'])
            for epoch in range(num_epochs):
                training_losses = []
                training_accs = []
                training_maes = []
                training_round_maes = []
                desc = 'Epoch ' + str(epoch) + '/' + str(num_epochs)
                for (x_train, y_train) in tqdm(dataloaders['train'], desc = desc):
                    try:
                        acc, loss, mae, round_mae = train_step(x_train, y_train, model, optimizer, criterion)
                    except ValueError as ve:
                        print(Fore.RED + ve + Fore.RESET)
                        yaml_dict = {'error': str(ve) + ' during train: exploding gradient!'}
                        with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
                            yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True)
                        return 0
                        
                    training_losses.append(loss)
                    training_accs.append(acc)
                    training_maes.append(mae)
                    training_round_maes.append(round_mae)
                average_training_loss = sum(training_losses) / len(training_losses)
                average_training_acc = sum(training_accs) / len(training_accs)
                average_training_mae = sum(training_maes) / len(training_maes)
                average_training_round_mae = sum(training_round_maes) / len(training_round_maes)
                average_training_losses.append(average_training_loss)
                average_training_accs.append(average_training_acc)
                average_training_maes.append(average_training_mae)
                average_training_round_maes.append(average_training_round_mae)

                # Evaluation
                val_accs = []
                val_loss = []
                val_maes = []
                val_round_maes = []
                for x_val, y_val in dataloaders['val']:
                    try:
                        acc, loss, mae, round_mae = val_step(x_val, y_val, model, criterion)
                    except ValueError as ve:
                        print(Fore.RED + ve + Fore.RESET)
                        yaml_dict = {'error': str(ve) + ' during val: exploding gradient!'}
                        with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
                            yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True)
                        return 0
                    
                    val_accs.append(acc)
                    val_loss.append(loss)
                    val_maes.append(mae)
                    val_round_maes.append(round_mae)
                average_val_loss = sum(val_loss) / len(val_loss)
                average_val_acc = sum(val_accs) / len(val_accs)
                average_val_mae = sum(val_maes)/len(val_maes)
                average_val_round_mae = sum(val_round_maes)/len(val_round_maes)
                average_val_accs.append(average_val_acc)
                average_val_losses.append(average_val_loss)
                average_val_maes.append(average_val_mae)
                average_val_round_maes.append(average_val_round_mae)
                # Meilleurs modèles
                # précision
                if best_val_accs < average_val_acc:
                    torch.save(model.state_dict(), os.path.join(results_dir, 'best.pt'))
                    best_val_accs = average_val_acc
                    best_epoch = epoch
                    best_epochs_steps.append(epoch)
                    print(Fore.GREEN + f"New best model: acc = {best_val_accs}", Fore.RESET)
                # précision et mae
                if best_val_accs == average_val_acc and best_mae_of_best_acc > average_val_mae:
                    torch.save(model.state_dict(), os.path.join(results_dir, 'best_and_mae.pt'))
                    best_mae_of_best_acc = average_val_mae
                    best_mae_of_best_acc_epoch = epoch
                    best_mae_of_best_acc_epochs_steps.append(epoch)
                    print(Fore.GREEN + f"New best mae for best acc model: acc = {best_val_accs}, mae = {best_mae_of_best_acc}", Fore.RESET)
                # précision et round mae
                if best_val_accs == average_val_acc and best_round_mae_of_best_acc > average_val_round_mae:
                    torch.save(model.state_dict(), os.path.join(results_dir, 'best_and_round_mae.pt'))
                    best_round_mae_of_best_acc =  average_val_round_mae
                    best_round_mae_of_best_acc_epoch = epoch
                    best_round_mae_of_best_acc_epochs_steps.append(epoch)
                    print(Fore.GREEN + f"New best round mae for best acc model: acc = {best_val_accs}, round mae = {best_round_mae_of_best_acc}", Fore.RESET)
                # round mae
                if (best_round_mae > average_val_round_mae) or (best_round_mae == average_val_round_mae and best_acc_of_round_mae < average_val_acc):
                    torch.save(model.state_dict(), os.path.join(results_dir, 'best_round_mae.pt'))
                    best_round_mae = average_val_round_mae
                    best_acc_of_round_mae = average_val_acc
                    best_round_mae_epoch = epoch
                    best_round_mae_epochs_steps.append(epoch)
                    print(Fore.GREEN + f"New best round mae model: round mae = {best_round_mae}, acc = {best_acc_of_round_mae}", Fore.RESET)
                # Early stopping
                if earlystop is not None:
                    if epoch > max(best_epoch, best_mae_of_best_acc_epoch, best_round_mae_of_best_acc_epoch, best_round_mae_epoch) + earlystop:
                        early_stopping = epoch
                        break
                writer.writerow([epoch, average_training_loss, average_training_acc, average_training_mae, average_training_round_mae, average_val_loss, average_val_acc, average_val_mae, average_val_round_mae])
                data = [{'epoch': epoch,
                         'train loss': average_training_loss,
                         'train mae': average_training_mae,
                         'train acc': average_training_acc,
                         'train round mae': average_training_round_mae,
                         'val loss': average_val_loss,
                         'val mae': average_val_mae,
                         'val acc': average_val_acc,
                         'val round mae': average_val_round_mae
                         }]
                print(Fore.WHITE + TablePrinter(fmt, ul='=')(data)+'\n'+'-'*180)

                # limite de temps
                end_t = time.time()
                time_elapsed=end_t - start_time
                if Time_limit is not None:
                    if str(datetime.timedelta(seconds=time_elapsed)) > str(datetime.timedelta(hours=Time_limit)):
                        early_stopping = epoch
                        break

        end_t = time.time()
        time_elapsed=end_t - start_time
        legend = ['train', 'val']
        print("time_elapsed: {}".format(str(datetime.timedelta(seconds=time_elapsed))))
        plt.style.use('dark_background')

        fig = plt.figure(layout="constrained", figsize=(20,10))
        gs = GridSpec(3, 1, figure=fig, wspace = 0.1)
        fig.suptitle('Results', fontsize=16)

        ax = fig.add_subplot(gs[0])
        intervals = np.arange(len(average_training_losses))
        ax.plot(intervals, average_training_losses, 'b')
        intervals = np.arange(1,len(average_training_losses))
        ax.plot(intervals, average_val_losses[1:], 'g')
        ax.set_ylabel("loss: " + criterion._get_name())

        ax = fig.add_subplot(gs[1])
        intervals = np.arange(len(average_training_accs))
        ax.plot(intervals, average_training_maes, 'b')
        intervals = np.arange(1,len(average_training_accs))
        ax.plot(intervals, average_val_maes[1:], color = 'g')
        ax.set_ylabel("loss: MAE")

        ax = fig.add_subplot(gs[2])
        intervals = np.arange(len(average_training_accs))
        ax.plot(intervals, average_training_accs, 'b')
        ax.plot(intervals, average_val_accs, color = 'g')
        
        ax.set_xlabel("epoch")
        ax.set_ylabel("accuracy")

        fig.legend(legend)
        fig.savefig(os.path.join(results_dir, 'results.png'))
        plt.close()
        if best_val_accs == 0 :
            print("Evaluation accuracy (best) = ", best_val_accs)
        elif best_val_accs > 0.9:
            print(Back.GREEN + "Evaluation accuracy (best) = ", best_val_accs)
        elif best_val_accs > 0.8:
            print(Back.BLUE + "Evaluation accuracy (best) = ", best_val_accs)
        elif best_val_accs > 0.65:
            print('\033[48;2;255;90;0m' + "Evaluation accuracy (best) = ", best_val_accs)
        else:
            print(Back.RED + "Evaluation accuracy (best) = ", best_val_accs)
        print(Back.RESET)

        # clear GPU cache memory
        torch.cuda.empty_cache()

        # Sauvegarde des résultats
        yaml_dict = {
                    '----Best Accuracy-----': '',
                    'time': str(datetime.timedelta(seconds=time_elapsed)),
                    'early_stopping': early_stopping,
                    'best acc epoch': best_epoch,
                    'best epochs steps': str(best_epochs_steps),
                    'best val acc': best_val_accs,
                    'best acc train loss': average_training_losses[best_epoch],
                    'best acc train acc': average_training_accs[best_epoch],
                    'best acc train mae loss': average_training_maes[best_epoch],
                    'best acc val loss': average_val_losses[best_epoch],
                    'best acc val mae loss': average_val_maes[best_epoch],
                    '-----Best MAE for Best Accuracy----': '',
                    'best mae of acc epoch': best_mae_of_best_acc_epoch,
                    'best mae of best acc steps': str(best_mae_of_best_acc_epochs_steps),
                    'best mae of best acc': best_mae_of_best_acc,
                    '-----Best Round MAE for Best Accuracy----': '',
                    'best round mae of acc epoch': best_round_mae_of_best_acc_epoch,
                    'best round mae of best acc epochs steps': str(best_round_mae_of_best_acc_epochs_steps),
                    'best round mae of best acc': best_round_mae_of_best_acc,
                    '----Best Round MAE-----': '',
                    'best round mae epoch': best_round_mae_epoch,
                    'best round mae epochs steps': str(best_round_mae_epochs_steps),
                    'best round mae': best_round_mae,
                    'best acc of bes round mae': best_acc_of_round_mae
                    }
        with open(os.path.join(results_dir, 'results.yaml'), 'w') as yamlf:
            yaml.dump(yaml_dict, yamlf, default_flow_style=False, allow_unicode=True, sort_keys=False)

    # Sauvegarde des matrices de confusion
    # charger le meilleur model
    model.load_state_dict(torch.load(os.path.join(results_dir, 'best.pt')))

    if not os.path.exists(os.path.join(results_dir, 'val_conf_mat.png')):
        path = os.path.join('dataset', 'val')
        conf_mat, inds_p, inds_t = get_confusion_matrix(model, path, data_transforms,2)
        inds_p = np.append(inds_p, 'other')
        df_cm = pd.DataFrame(conf_mat, index = [i for i in inds_p],
                        columns = [i for i in inds_t])
        plt.figure(figsize = (10,7))
        sn.heatmap(df_cm, annot=True)
        plt.ylabel('predicted')
        plt.xlabel('ground truth')
        plt.title('Val confusion matrix')
        plt.savefig(os.path.join(results_dir, 'val_conf_mat.png'))
        plt.close()

    # train
    if not os.path.exists(os.path.join(results_dir, 'train_conf_mat.png')):
        path = os.path.join('dataset', 'train')
        conf_mat, inds_p, inds_t = get_confusion_matrix(model, path, data_transforms,2)
        inds_p = np.append(inds_p, 'other')
        df_cm = pd.DataFrame(conf_mat, index = [i for i in inds_p],
                        columns = [i for i in inds_t])
        plt.figure(figsize = (10,7))
        sn.heatmap(df_cm, annot=True)
        plt.ylabel('predicted')
        plt.xlabel('ground truth')
        plt.title('Train confusion matrix')
        plt.savefig(os.path.join(results_dir, 'train_conf_mat.png'))
        plt.close()

    return best_val_accs


In [ ]:
def Test_set(seed, params:dict):
    """Entraine un modèle avec un set de paramètres

    Args:
        seed (int): Graine d'aléatoire
        params (dict): Parmètres

    Returns:
        str, float: Chemin des résultats et meilleure précision
    """
    # Réinitialiser les seeds RNG
    if Same_model:
        torch.cuda.manual_seed(Model_gen_seed)
        torch.manual_seed(Model_gen_seed)
    else:
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)

    # dossier cible
    results_dir=os.path.join(seed_results_dir, get_path(params), f'seed_{seed}')
    os.makedirs(results_dir, exist_ok=True)
    # Sauvegarde des paramètres
    with open(os.path.join(results_dir, 'args.yaml'), 'w') as yamlf:
        yaml.dump(params, yamlf, default_flow_style=False, allow_unicode=True, sort_keys=False)

    # Génération du modèle
    model = gen_new_model(params['freeze'], bias_init_value)
    # seed pour dataloaders
    if Same_shuffle:
        torch.cuda.manual_seed(Shuffle_seed)
        torch.manual_seed(Shuffle_seed)
    else:
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device) # déplacer le model vers le GPU a faire avant de construire l'optimiseur (pour adagrad)
    optimizer = get_optimizer(model, params)
    criterion = get_criterion(params)
    dataloaders, data_transforms = get_dataloaders(params)
    
    # Sauvegarde de l'architecture du modèle
    model_stats = summary(model, input_size=(params['batch_size'], 3, 224, 224), row_settings=("depth", "ascii_only"))
    summary_str = str(model_stats)
    with open(os.path.join(results_dir, 'model.txt'), 'w') as modelf:
        modelf.write(summary_str)

    # entrainement
    train_params = {
        'model': model,
        'epochs': params['epochs'],
        'earlystop': params['earlystop'],
        'results_dir': results_dir,
        'dataloarders': dataloaders,
        'optimizer': optimizer,
        'criterion': criterion,
        'data_transforms': data_transforms
    }
    best_acc = training_loop(train_params)
    print(results_dir)
    return results_dir, best_acc

In [ ]:
def SeedsTrain(params, seeds:list, results_pdf):
    """Boucle d'entrainement des modèles pour chaque graine

    Args:
        params (Dict): Paramètres
        seeds (list): Liste des graines
        results_pdf (str): Chemin du PDF
    """
    # init
    results = [('Seed', 'Best Accuracy')]
    i = 0 # seulement pour l'affichage
    for seed_id, seed in enumerate(seeds):
        i+=1
        if i%5==0:
            clear_output() # otherwise too many outputs
        print(Fore.RED + '=========================================================={}/{}=========================================================='.format(seed_id, len(seeds)))
        results_dir, best_acc = Test_set(seed, params)
        results.append((results_dir, best_acc))
    print(results)
    gen_pdf(results=results, path=os.path.join(seed_results_dir, results_pdf))
    return

In [ ]:
SeedsTrain(params, seeds, results_pdf)